In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pynput.keyboard import Controller, Key

keyboard = Controller()

def run_success_keybinding(keyboard: Controller):
    keyboard.press(Key.cmd)
    keyboard.press(Key.shift)
    keyboard.press(Key.alt)
    keyboard.press('g')

    keyboard.release(Key.cmd)
    keyboard.release(Key.shift)
    keyboard.release(Key.alt)
    keyboard.release("g")


def run_failed_keybinding(keyboard: Controller):
    keyboard.press(Key.cmd)
    keyboard.press(Key.shift)
    keyboard.press(Key.alt)
    keyboard.press("b")

    keyboard.release(Key.cmd)
    keyboard.release(Key.shift)
    keyboard.release(Key.alt)
    keyboard.release("b")


run_success_keybinding(keyboard)
run_failed_keybinding(keyboard)

In [8]:
run_failed_keybinding(keyboard)

In [3]:
Javascript(
    """
const vscode = require('vscode');
consolte.log("Test direct from Jupyter Notebook!");
vscode.window.showWarningMessage("Test direct from Jupyter Notebook!");

}
"""
)

NameError: name 'Javascript' is not defined

In [5]:
console.log("Test direct from Jupyter Notebook!");

NameError: name 'console' is not defined